In [38]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

RANDOM_SEED = 42

# Specify each path

In [39]:
dataset = 'model/static_classifier/static_data.csv'
model_save_path = 'model/static_classifier/static_classifier.hdf5'
tflite_save_path = 'model/static_classifier/static_classifier.tflite'

# Set number of classes

In [40]:
NUM_CLASSES = 6

# Dataset reading

In [41]:
static_gestures = [
    'open',
    'closed',
    'pointer',
    'ok',
    'peace',
    'thumbs up'
]

data = np.concatenate([
    np.load('dataset/raw_closed_1632795128.npy'),
    np.load('dataset/raw_ok_1632795128.npy'),
    np.load('dataset/raw_open_1632795128.npy'),
    np.load('dataset/raw_peace_1632795128.npy'),
    np.load('dataset/raw_pointer_1632795128.npy'),
    np.load('dataset/raw_thumbs up_1632795128.npy')
], axis=0)

In [42]:
X_dataset = data[:, :-1]
X_dataset.shape

(1226, 99)

In [43]:
labels = data[:, -1]
y_dataset = to_categorical(labels, num_classes=len(static_gestures))
y_dataset.shape

(1226, 6)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)
print(X_train.shape)
print(y_train.shape)

(919, 99)
(919, 6)


# Model building

In [45]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((99, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(40, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [46]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_6 (Dropout)          (None, 99)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 40)                4000      
_________________________________________________________________
dropout_7 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_11 (Dense)             (None, 6)                 126       
Total params: 4,946
Trainable params: 4,946
Non-trainable params: 0
_________________________________________________________________


In [47]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [48]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Model training

In [49]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
8/8 [==============================] - 0s 22ms/step - loss: 26.6962 - accuracy: 0.2331 - val_loss: 12.3250 - val_accuracy: 0.2443

Epoch 00001: saving model to model/static_classifier\static_classifier.hdf5
Epoch 2/1000
8/8 [==============================] - 0s 5ms/step - loss: 19.8192 - accuracy: 0.2300 - val_loss: 10.0050 - val_accuracy: 0.2052

Epoch 00002: saving model to model/static_classifier\static_classifier.hdf5
Epoch 3/1000
8/8 [==============================] - 0s 5ms/step - loss: 14.1248 - accuracy: 0.2490 - val_loss: 6.5632 - val_accuracy: 0.2182

Epoch 00003: saving model to model/static_classifier\static_classifier.hdf5
Epoch 4/1000
8/8 [==============================] - 0s 5ms/step - loss: 12.4112 - accuracy: 0.2318 - val_loss: 4.3669 - val_accuracy: 0.2671

Epoch 00004: saving model to model/static_classifier\static_classifier.hdf5
Epoch 5/1000
8/8 [==============================] - 0s 5ms/step - loss: 8.7608 - accuracy: 0.2997 - val_loss: 3.0339 - val_ac


Epoch 00038: saving model to model/static_classifier\static_classifier.hdf5
Epoch 39/1000
8/8 [==============================] - 0s 5ms/step - loss: 0.9996 - accuracy: 0.6867 - val_loss: 0.5011 - val_accuracy: 0.9055

Epoch 00039: saving model to model/static_classifier\static_classifier.hdf5
Epoch 40/1000
8/8 [==============================] - 0s 4ms/step - loss: 1.0177 - accuracy: 0.6743 - val_loss: 0.5105 - val_accuracy: 0.9088

Epoch 00040: saving model to model/static_classifier\static_classifier.hdf5
Epoch 41/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.9909 - accuracy: 0.6782 - val_loss: 0.5155 - val_accuracy: 0.9153

Epoch 00041: saving model to model/static_classifier\static_classifier.hdf5
Epoch 42/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.8897 - accuracy: 0.7240 - val_loss: 0.5103 - val_accuracy: 0.9153

Epoch 00042: saving model to model/static_classifier\static_classifier.hdf5
Epoch 00042: early stopping


In [50]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

3/3 [==============================] - 0s 997us/step - loss: 0.5103 - accuracy: 0.9153


In [51]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [52]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

[0.7916097  0.01725816 0.10347933 0.03418686 0.04261157 0.01085438]
0


# Confusion matrix

In [55]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
    ax.set_ylim(len(set(y_true)), 0)
    plt.show()
    
    if report:
        print('Classification Report')
        print(classification_report(y_test, y_pred))

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, y_pred)

TypeError: unhashable type: 'numpy.ndarray'

# Convert to model for Tensorflow-Lite

In [56]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)

In [57]:
# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: C:\Users\jerem\AppData\Local\Temp\tmp4aw7pp4g\assets


9936

# Inference test

In [142]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [143]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [144]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [145]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

Wall time: 0 ns


In [146]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[0.1548571  0.12690188 0.6782096  0.02090419 0.0118072  0.00731999]
2
